In [77]:
import gym
import numpy as np
import tensorflow as tf
import cv2
import random

In [78]:
# Implementing PER
class SumTree:
    # Visualised in the form a binray heap
    
    def __init__(self,size):
        self.leaves = size # Number of leaf nodes in the sum tree
        self.size = 2*size - 1 # Number of nodes in the sum tree
        self.data_pointer = 0
        self.tree = np.zeros((self.size))
        self.data = np.zeros((self.size),dtype=object) # It stores an experience tuple
        
    def add(self,priority,data):
        tree_index = self.data_pointer + self.leaves - 1 # Refers to the leaf node number indexed by data_pointer

        self.data[self.data_pointer] = data
        
        self.update(tree_index,priority)
        
        #Updating to the next index
        self.data_pointer+=1
        
        # Checking for overflow
        if self.data_pointer >= self.leaves:
            self.data_pointer = 0
    
    def update(self,tree_index,priority):
        # print('In Update---')
        # print('Tree Index : ' + str(tree_index))
        # print('Priority : ' + str(priority))
        self.tree[tree_index] = priority
        
        # Update the non-leaf nodes up the tree
        parent = ((tree_index + 1)//2) - 1

        while parent >= 0:
            parent = ((tree_index + 1)//2) - 1
            
            if parent < 0:
                break

            left_child = 2*parent + 1
            right_child = left_child + 1
            self.tree[parent] = self.tree[left_child] + self.tree[right_child]
            tree_index = parent

    def get_leaf(self,s):
        # Return the leaf node such that the cumulative priority till that is <= s
        # Values are returned as index,data,priority
        # In short this 
        index = 0 # Start from the root
        leaf_index = None
        
        while True:
            left_child = 2*index + 1
            right_child = left_child + 1
            
            if left_child >= self.size: # index is the leaf node
                leaf_index = index
                break
            else:
                if self.tree[left_child] >= s:
                    index = left_child
                else:
                    s = s - self.tree[left_child]
                    index = right_child
        
        return leaf_index,self.data[leaf_index - self.leaves + 1],self.tree[leaf_index]
    
    def get_total_priority(self):
        return self.tree[0] # The root value

In [79]:
class PER:
    def __init__(self,size):
        self.size = size
        self.tree = SumTree(size)
        
        # PER constants
        self.alpha = 0.4
        self.beta = 0.6
        self.beta_increment_factor = 0.001
        self.epsilon = 0.01
        
        self.maximum_absolute_error = 1.0 # To clip the TD errors
        
    def store(self,experience):
        # experience : a tuple of (s,a,r,s')
        max_priority = np.max(self.tree.tree[-self.tree.leaves:])

        if max_priority == 0:
            max_priority = 1.0 # Add any random value for now as it will in due course be updated
        
        # We use maximum  priority for a new experience to ensure that it will be use at least once
        # This breaks the bias
        self.tree.add(max_priority,experience)
    
    def sample_minibatch(self,k):
        '''
        Parameters:
        k : mini-batch size
        
        Returns:
        batch_idx : index values of the SumTree of the nodes which were sampled so that their errors can be updated
        mini_batch : array of experience tuples in the mini-batch
        w : importance sampling weights
        '''
        
        # Samples a minibatch of size k
        mini_batch = []
        batch_idx = np.zeros((k))
        
        total_priority = self.tree.get_total_priority()
        
        # Divide the total probability in bins
        num_bins = total_priority/k
        
        # Importance Sampling Weights
        w = np.zeros((k,1))
        
        P_min = np.min(self.tree.tree[-self.tree.leaves:])/self.tree.get_total_priority()
        # print('P_min : ' + str(P_min))
        max_w = (1.*k*P_min)**(-self.beta)
        # print('Max w : ' + str(max_w))
        
        for i in range(k):
            # Find the bin priority indices
            left_index = i*num_bins
            right_index = (i+1)*num_bins
            s = np.random.uniform(left_index,right_index)
            
            # Sample a random node with priority in range [i*num_bins,(i+1)*num_bins]
            # This samples a node from a probability distribution represented by the priorities
            idx,data,priority = self.tree.get_leaf(s)
            
            mini_batch.append(data)
            
            # Calculate P(i)
            # We assume that the values in the tree are already exponentiated while updation
            # i.e the tree leaves store (p(i))**alpha
            sampling_probability = priority/self.tree.get_total_priority()
            
            # Calculate the weights
            # print('Priority : ' + str(priority))
            # print('Sampling Probability : ' + str(sampling_probability))
            # print('Calculated weight : ' + str(np.power(1.*k*sampling_probability,-self.beta)/(max_w)))
            w[i,0] = (1.*k*sampling_probability)**(-self.beta)/(max_w)
            
            # Set the index of the tree leaf
            batch_idx[i] = idx
        
        # Increment beta at each sampling step
        self.beta = np.min([1.,self.beta + self.beta_increment_factor])
        
        return batch_idx.astype(np.int),mini_batch,w
    
    def update(self,batch_idx,errors):
        '''
        errors : TD errors
        Updates the indices which were sampled with their corrected error values
        '''
        
        absolute_errors = np.abs(errors) + self.epsilon
        
        # Store (p(i))**alpha in the tree leaves
        clipped_errors = np.minimum(absolute_errors,self.maximum_absolute_error) # Clipped in range [0,1]
        exponentiated_errors = np.power(clipped_errors,self.alpha)
        
        # print('In Loop')
        for i,delta in zip(batch_idx,exponentiated_errors):
            # print(i)
            # print(delta)
            self.tree.update(i,delta)
    
    def print_priorities(self):
        print('Number of experiences stored : ' + str(len(self.tree.tree[-self.tree.leaves:])))
        print('Priorities : ' + str(self.tree.tree[-self.tree.leaves:]))

In [80]:
def test_PER():
    mem_size = 100
    nA = 2 # Number of actions

    memory = PER(mem_size)

    env.reset()
    for _ in range(mem_size):
        state = env.env.state
        action = env.action_space.sample()
        next_state,reward,done,_ = env.step(action) # take a random action
        if done:
            env.reset()
        memory.store((state,action,reward,next_state))
    
    memory.print_priorities()
    batch_idx,min_batch,w = memory.sample_minibatch(7)
    print(batch_idx)
    errors = np.random.randn(5)
    print(errors)
    print(w)
    memory.update(batch_idx,errors)
    memory.print_priorities()

In [81]:
class DQNAgent:
    def __init__(self,state_size,action_size,memory_size=100000,learning_rate=1e-5,name='DQNAgent'):
        self.state_size = state_size
        self.action_size = action_size
        self.memory_size = memory_size
        self.learning_rate = learning_rate
        self.name = name
        
        self.build_model()
    
    def build_model(self):
        with tf.variable_scope(self.name,reuse=tf.AUTO_REUSE):
            '''
            Our state consists of four stacked frames of the game of shape - (WIDTH,HEIGHT,4)
            '''
            # For the Importance Sampling Weights
            self.weights_placeholder = tf.placeholder(shape=[None,1],name='weights_placeholder',dtype=tf.float32)
            self.state = tf.placeholder(shape=[None,*self.state_size],name='state',dtype=tf.float32)
            
            # actions_ is a one hot vector
            self.actions_ = tf.placeholder(shape=[None,self.action_size],name='actions_',dtype=tf.float32) 
            
            # Our target Q value
            self.target_Q = tf.placeholder(shape=[None],name='target_Q',dtype=tf.float32)

            self.conv1 = tf.nn.elu(tf.layers.conv2d(self.state,filters=32,kernel_size=[4,4],\
                                                     strides=[4,4],name='conv1'),name='elu1')
            self.conv2 = tf.nn.elu(tf.layers.conv2d(self.conv1,filters=64,kernel_size=[4,4],\
                                                     strides=[2,2],name='conv2'),name='elu2')
            self.conv3 = tf.nn.elu(tf.layers.conv2d(self.conv2,filters=128,kernel_size=[2,2],\
                                                     strides=[2,2],name='conv3'),name='elu3')
            self.flat = tf.layers.flatten(self.conv3)
            
            # Branch out now to V(s) and A(s,a)
            self.V_fc = tf.layers.dense(self.flat,128,kernel_initializer=tf.contrib.layers.xavier_initializer()\
                                       ,activation = tf.nn.elu,name='V_fc')
            self.V = tf.layers.dense(self.V_fc,1,kernel_initializer=tf.contrib.layers.xavier_initializer()\
                                    ,name='V')
            
            self.A_fc = tf.layers.dense(self.flat,128,kernel_initializer=tf.contrib.layers.xavier_initializer()\
                                       ,activation = tf.nn.elu,name='A_fc')
            self.A = tf.layers.dense(self.A_fc,self.action_size,kernel_initializer=tf.contrib.layers.xavier_initializer()\
                                    ,name='A')
            
            # This stores the Q values for all possible actions
            self.Q_all_a = self.V + (self.A - tf.reduce_mean(self.A,axis=1,keepdims=True))
            
            # This keeps the values only for the action taken - Q[s,a] in the transition tuple
            self.Q = tf.reduce_sum(tf.multiply(self.Q_all_a,self.actions_),axis=1)
            
            self.TD_error = tf.abs(self.target_Q - self.Q)

            self.loss = tf.reduce_mean(self.weights_placeholder*tf.square(self.target_Q - self.Q))
            
            self.optimizer = tf.train.RMSPropOptimizer(learning_rate=self.learning_rate).minimize(self.loss)

In [82]:
from collections import deque

class StateProcessor(object):
    '''
    State Processor for the Breakout-v0 environment
    '''
    def __init__(self,name='Breakout-v0'):
        self.name = name
        self.env = gym.make(name)
        self.stack_size = 4
        self.reset()
    
    def reset(self):
        self.is_new_episode = True
        self.state = self.env.reset()        
        self.state_stack  =  deque([np.zeros((84,84), dtype=np.int) for i in range(self.stack_size)], maxlen=4)
        self.stack_states()
    
    def stack_states(self):
        '''
        Stack the 4 previous frames
        '''
        
        if self.is_new_episode is True:
            self.state_stack.append(self.process(self.state))
            self.state_stack.append(self.process(self.state))
            self.state_stack.append(self.process(self.state))
            self.state_stack.append(self.process(self.state))
            
            self.is_new_episode = False
        else:
            self.state_stack.append(self.process(self.state))           
        
        self.stacked_frames = np.stack(self.state_stack,axis=2)
            
    def process(self,state):
        '''
        Takes as input a (210,160,3) image
        Processes it to (84,80,1) image
        '''
        image = np.array(state,dtype=np.uint8)
        return np.mean(image[::2,::2],axis=2).astype(np.uint8)[14:98,:]
        
    def step(self,action):
        self.state,reward,done,_ = self.env.step(action)
        self.stack_states()
        return self.stacked_frames,reward,done
    
    def get_state(self):
        return self.stacked_frames
    
    def sample_env_action(self):
        return self.env.action_space.sample()
    
    def render(self):
        env.render()

In [109]:
# Main function
# Let's create all the stuff now

# Constants
state_size = [84,80,4]
action_size = 4
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.99
mem_size = 100
batch_size = 32
num_episodes = 1000
episode_length = 1000 # End an episode after these steps
network_copy_frequency = 10000
network_copy_frequency_step = 0

# Variables
processor = StateProcessor()
memory = PER(mem_size)

# Variables to save
reward_holder = []

tf.reset_default_graph()
sess = tf.Session()

agentNetwork = DQNAgent(state_size,action_size,'Agent')
targetNetwork = DQNAgent(state_size,action_size,'Target')

# # Tensorboard visualiser
# writer = tf.summary.FileWriter('/tmp/breakout/1')
# writer.add_graph(sess.graph)

def choose_action(state,epsilon,epsilon_decay):
    '''
    Choose epsilon greedy action
    '''
    rand = np.random.rand(1)[0]
    
    if rand < epsilon:
        # Select a random action
        action = random.randint(0,action_size - 1)
        epsilon*=epsilon_decay
    else:
        # Select a = argmax_a(Q[s,a])
        Q = sess.run([agentNetwork.Q_all_a],feed_dict={agentNetwork.state:state.reshape((1,*state_size))})
        action = np.argmax(Q)
    
    return action,epsilon

# Fill our memory with dummy values
for i in range(mem_size):
        # print(i)
        state = processor.get_state()
        action = processor.sample_env_action()
        next_state,reward,done = processor.step(action) # take a random action
        if done:
            processor.reset()
        memory.store((state,action,reward,next_state,done))  

# This function helps us to copy one set of variables to another
# In our case we use it when we want to copy the parameters of DQN to Target_network
# Thanks of the very good implementation of Arthur Juliani https://github.com/awjuliani
def update_target_graph():
    
    # Get the parameters of our DQNNetwork
    from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Agent")
    
    # Get the parameters of our Target_network
    to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Target")
    
    op_holder = []
    
    # Update our target_network parameters with DQNNetwork parameters
    for from_var,to_var in zip(from_vars,to_vars):
        op_holder.append(to_var.assign(from_var))
    return op_holder

In [120]:
sess.run(tf.global_variables_initializer())
print('Done')
sess.run(tf.local_variables_initializer())

saver = tf.train.Saver()

for episode in range(num_episodes):
    state = processor.reset()
    
    total_rewards = 0.0
    
    # Collect experience
    for _ in range(episode_length):
        state = processor.get_state()
        
        network_copy_frequency_step+=1
        
        action,epsilon = choose_action(state,epsilon,epsilon_decay)
        
        next_state,reward,done = processor.step(action)
        
        if done:
            memory.store((state,action,reward,np.zeros((state_size[0],state_size[1],state_size[2]))\
                          ,done)) # No next state is there 
            total_rewards+=reward
            break
        
        total_rewards+=reward
        
        memory.store((state,action,reward,next_state,done)) 
    
    # Train our agent
    
    # Sample a minibatch of transition tuples
    batch_idx,min_batch,w = memory.sample_minibatch(batch_size)
    
    # Seperate out our mini-batch variables
    states = np.array([min_batch[i][0] for i in range(len(min_batch))])
    actions = np.array([min_batch[i][1] for i in range(len(min_batch))])
    rewards = np.array([min_batch[i][2] for i in range(len(min_batch))]).reshape(-1)
    next_states = np.array([min_batch[i][3] for i in range(len(min_batch))])
    dones = np.array([min_batch[i][4] for i in range(len(min_batch))])
    
    print(next_states.shape)
    
    Qs_next_state,actions_next_state = sess.run([agentNetwork.Q_all_a,tf.argmax(agentNetwork.Q_all_a,axis=1)]\
                             ,feed_dict={agentNetwork.state:next_states})
    
    # Convert actions in current state of transition tuple to a one hot vector
    actions_one_hot_batch = np.zeros((batch_size,action_size))
    for i in range(batch_size):
        actions_one_hot_batch[i,actions[i]] = 1
    
    # Convert actions in next state to a one hot vector
    actions_ns_one_hot_batch = np.zeros((batch_size,action_size))
    for i in range(batch_size):
        actions_ns_one_hot_batch[i,actions_next_state[i]] = 1
        
    target_Q_next_state = sess.run([targetNetwork.Q],feed_dict={targetNetwork.state:next_states,\
                                                               targetNetwork.actions_:actions_ns_one_hot_batch})

    target_Q = np.zeros(batch_size)
    
    for i in range(batch_size):
        if dones[i] is True:
            target_Q[i] = rewards[i]
        else:
            target_Q[i] = rewards[i] + gamma*target_Q_next_state[0][i]
    
    # Make optimizer updates
    _,loss,TD_errors = sess.run([agentNetwork.optimizer,agentNetwork.loss,agentNetwork.TD_error],\
                     feed_dict={agentNetwork.state:states,agentNetwork.actions_:actions_one_hot_batch\
                               ,agentNetwork.target_Q:target_Q,agentNetwork.weights_placeholder:w})
    
    # Update the priorites in the memory
    memory.update(batch_idx,TD_errors)
    
    print('Episode : ' + str(episode) + '\nTotal Rewards : ' + str(total_rewards) + '\nLoss : ' + str(loss))
    
    if network_copy_frequency_step > network_copy_frequency:
        network_copy_frequency_step = 0
        update_op = update_target_graph()
        sess.run(update_op)
        print('Model Updated')

Done
(32, 84, 80, 4)
Episode : 0
Total Rewards : 0.0
Loss : 0.0037748755
(32, 84, 80, 4)
Episode : 1
Total Rewards : 0.0
Loss : 0.81870985
(32, 84, 80, 4)
Episode : 2
Total Rewards : 0.0
Loss : 0.15333778
(32, 84, 80, 4)
Episode : 3
Total Rewards : 1.0
Loss : 56.06001
(32, 84, 80, 4)
Episode : 4
Total Rewards : 1.0
Loss : 0.0008460151
(32, 84, 80, 4)
Episode : 5
Total Rewards : 1.0
Loss : 0.0007177124
(32, 84, 80, 4)
Episode : 6
Total Rewards : 0.0
Loss : 0.000598622
Model Updated
(32, 84, 80, 4)
Episode : 7
Total Rewards : 0.0
Loss : 0.000496122
(32, 84, 80, 4)
Episode : 8
Total Rewards : 1.0
Loss : 0.00041567543
(32, 84, 80, 4)
Episode : 9
Total Rewards : 1.0
Loss : 0.00033912036
(32, 84, 80, 4)
Episode : 10
Total Rewards : 0.0
Loss : 0.42863977
(32, 84, 80, 4)
Episode : 11
Total Rewards : 0.0
Loss : 0.00055429013
(32, 84, 80, 4)
Episode : 12
Total Rewards : 0.0
Loss : 0.00043689067
(32, 84, 80, 4)
Episode : 13
Total Rewards : 0.0
Loss : 0.0003397699
(32, 84, 80, 4)
Episode : 14
Tota

KeyboardInterrupt: 

In [118]:
a = np.zeros((state_size[0],state_size[1],state_size[2]))

In [119]:
a.shape

(84, 80, 4)